### 데이터 불러오기

파일의 이름에 숫자만 다른 것을 이용하여 반복문으로 한번에 데이터를 가져와 df에 할당합니다.

In [2]:
echo "# OUTTA_Boot_Camp" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/leejoon2067/OUTTA_Boot_Camp.git
git push -u origin main

SyntaxError: invalid syntax (4219063710.py, line 1)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import re
warnings.filterwarnings(action='ignore')

#일반적으로 사용하는 데이터 파일 불러오는 방법
df= pd.read_excel("raw_data.xlsx")
df

In [ ]:
#반복문을 이용해서 데이터 파일 data에 저장

#data=[]
#for i in range(1,6):
    #data.함수1(pd.read_excel(f"파일명규칙활용.xlsx"))    
#    data.add(pd.read_excel(f"C:/Users/leejo/Downloads/outta 부트캠프/제공데이터/제공데이터/rank1.xlsx")
    
#함수2를 이용하여 data에 있는 데이터들을 합치기
#df = pd.함수2(data, ignore_index=True)   
#df.head()

### 결측값 제거

크롤링 코드 작성 시 크롤링하지 못한 정보는 '없음'으로 저장하게 했기 때문에 '없음'의 갯수로 결측값의 갯수를 파악합니다.

In [ ]:
###############제공된 코드 건드리지 마세요 ###################

### 데이터 컬럼별 없음(결측치) 개수 파악

col_name= df.columns    #컬럼명 저장
from collections import Counter

for i in col_name:
    print(i,Counter(df[i])["없음"])
    

In [ ]:
#피부정보, 별점, 피부타입, 피부고민, 자극도 중 하나라도 값이 '없음'인 경우 
df_new = df[(df['피부정보'] == '없음') | (df['별점'] == '없음')| (df['피부타입'] == '없음')| (df['자극도'] == '없음')]
df_new

In [ ]:
### 별점, 피부타입, 피부고민, 자극도 값이 없음은 제거한 데이터를 df2에 저장

df2= df[~df.index.isin(df_new.index)]
df2.reset_index(drop=True, inplace=True)
df2.shape

In [ ]:
###############제공된 코드 건드리지 마세요 ###################
### 데이터 컬럼별 없음(결측치) 개수 파악

col_name= df2.columns    #컬럼명 저장
from collections import Counter

for i in col_name:
    print(i,Counter(df2[i])["없음"])

### 정가, 할인가 데이터 타입 변경

정가,할인가의 쉼표를 제거하고 데이터 타입을 int64로 바꿔줍니다.

In [ ]:
# df2['정가'] = df2['정가'].str.replace(',', '').astype('int64')
# df2['할인가'] = df2['할인가'].str.replace(',' , '').astype('int64')

# df2.head()

In [ ]:
df2.dtypes

### 아이디 컬럼 삭제

모델 학습 시 아이디 정보는 필요하지 않기 때문에 제거해줍니다.

In [ ]:
df2 = df2.drop(columns = ['아이디'])
df2.head()

### 별점에서 숫자만 남기기

현재 별점은 5점만점에 X점과 같은 형식으로 되어있습니다. 여기서 X만 남기고 데이터 타입을 int64로 바꿔줍니다.

In [ ]:
df2['별점'] = df2['별점'].apply(lambda x: int(re.findall(r'\d+',x)[1]))

df2.head()

In [ ]:
df2.dtypes

### 피부정보 컬럼을 여러컬럼으로 구분

피부정보에는 최대 피부타입, 피부톤, 2개의 피부고민 총 4개로 이루어져 있습니다.  
해당 텍스트는 공백으로 구분되어있고 이를 이용해 4개의 컬럼(user피부타입/user피부톤/user피부고민1/user피부고민2)으로 만들어 줍니다.

In [ ]:
#힌트: split 함수
df2[["user피부타입", "user피부톤", "user피부고민1", "user피부고민2"]] = df2["피부정보"].str.split(expand=True)

df2

#### user피부타입 컬럼에서 복합성/건성/지성/민감성/트러블성/약건성/중성 을 제외하고 모두 np.NaN 처리 해줍니다.  

In [ ]:
#user피부타입에서 해당하지 않는 값은 nan 처리

types = ['복합성', '건성', '지성', '민감성', '트러블성', '약건성', '중성']
def to_nan(x):
    if x not in types:
        return np.NaN
    else:
        return x
        
        
df2["user피부타입"] = df2['user피부타입'].apply(to_nan)
df2["user피부타입"].value_counts()

#### user피부고민2의 결측값을 user피부톤 컬럼값으로 채운 후  (웜톤/쿨톤/여름쿨톤/봄웜톤/가을웜톤/겨울쿨톤) 값을  np.NaN 처리 해줍니다.

In [ ]:
def fill_skin_tone(row):
    if pd.isna(row['user피부고민2']):
        return row['user피부톤']
    else:
        return row['user피부고민2']


df2["user피부고민2"]= df2.apply(fill_skin_tone, axis =1)
df2["user피부고민2"].value_counts()

In [ ]:
tones = ['웜톤', '쿨톤', '여름쿨톤', '봄웜톤', '가을웜톤', '겨울쿨톤']

def to_nan_피부고민2(x):
    if x in tones:
        return np.NaN
    else:
        return x
        
        
df2["user피부톤"] = df2['user피부톤'].apply(to_nan_피부고민2)
df2["user피부고민2"].value_counts()

### user피부타입, usesr피부고민1, user피부고민2 에서 결측값 제거

In [ ]:
df2.isnull().sum()

In [ ]:
df2.dropna(subset = ['user피부타입', 'user피부고민1', 'user피부고민2'], inplace = True)
df2.isnull().sum()

In [ ]:
df2.head()

In [ ]:
df2.shape

### df2를 df3에 복사한 후 상품명컬럼의 값을 간단하게 변경해줍니다.
* 대용량 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+31ml 증정 기획                                     : goodal
* [1등 국민미스트] 달바 화이트 트러플 퍼스트 스프레이 세럼 100ml + 100ml 기획세트                        : dalba_mist
* 성분에디터 그린토마토 포어 리프팅 앰플 플러스 30ml+10ml 기획                                           : sungboon
* [2세대 천만크림] 닥터지 레드 블레미쉬 클리어 수딩 크림 70ml+30ml 세트                                  : dr_g
* 라로슈포제 NEW 시카플라스트 밤B5+ 100ml 기획(+시카토너 50ml 증정)                                      : larocheposay
* [100ml+100ml/단독기획] 토리든 다이브인 저분자 히알루론산 수딩크림 더블기획                             : torriden_cream
* 에스트라 아토베리어365 크림 80ml 기획 (+하이드로에센스 25ml+무기자차선크림10ml 증정)(2305)             : aestura_cream
* 에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선크림10ml 증정)                  : aestura_lotion
* [단독기획] 토리든 다이브인 저분자 히알루론산 세럼 더블기획 (50ml+50ml)                                 : torriden_serum
* [5월 올영픽/대용량] 파티온 노스카나인 트러블 세럼 단독 기획(50ml+15ml)                                 : fation
* [쿵야 키링&스티커 증정] V&A 안티옥시던트 래디언스 앰플 더블 기획 (30ml+30ml)                           : V&A
* [박은빈 PICK] 한율 어린쑥 수분진정크림 55ml 기획(+45ml 리필 증정)                                      : hanyul

In [ ]:
df3=df2.copy()


#상품명 간소화
def name_min(x):
    if "[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획" in x:
        return "goodal_serum"
    elif "[8월 올영픽/1+1단독] 성분에디터 그린토마토 포어 리프팅 앰플 플러스 30ml 더블 기획" in x:
        return "green_tomato_ampule"
    elif "[뽐니PICK/한정기획] 토리든 다이브인 세럼 50ml 리필 한정 기획(+50ml 리필팩+수딩 크림 20ml)" in x:
        return "torriden_serum"
    elif "[8월 올영픽/소의튜브 PICK] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 기획 (60ml+30ml)" in x:
        return "snature_cream"
    elif "[8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ml 리필 기획세트(350ml+350ml리필)" in x:
        return "anua_toner"
    else:
        return x

# 결과 출력
df3["상품명"]= df3["상품명"].apply(name_min)
df3.head()

In [ ]:
df3.shape

### 브랜드, 피부정보, user피부톤 컬럼을 삭제합니다.

In [ ]:
df3 = df3.drop(columns = ['브랜드'])
df3 = df3.drop(columns =['피부정보'])
df3 = df3.drop(columns = ['user피부톤'])

In [ ]:
df3.shape

### 컬럼명을 변경해줍니다.
* 피부타입 -> 효과1  
* 피부고민 -> 효과2  
* user피부타입 -> 피부타입  
* user피부고민1 -> 피부고민1  
* user피부고민2 -> 피부고민2 

In [ ]:
df3.rename(columns = {
    "피부타입": "효과1",
    "피부고민": "효과2",
    "user피부타입": "피부타입",
    "user피부고민1": "피부고민1",
    "user피부고민2": "피부고민2"
}, inplace = True)
df3.head()

In [ ]:
df3.shape

## 데이터 인코딩

In [ ]:
df3["자극도"].value_counts()

### 직접 함수를 만들어 자극도 컬럼을 레이블인코딩 합니다.
* 자극없이 순해요 -> 0
* 보통이에요 -> 1
* 자극이 느껴져요 -> 2

In [ ]:
def 자극도인코딩(x):
    if x == "자극없이 순해요":
        return 0
    elif x == "보통이에요":
        return 1
    elif x == "자극이 느껴져요":
        return 2
    else:
        return x
    

df3["자극도"]= df3["자극도"].apply(자극도인코딩)
df3["자극도"].value_counts()

### 상품명, 카테고리, 효과1, 효과2, 피부타입, 피부고민1, 피부고민2 은  get_dummies를 이용해 원핫인코딩을 합니다.
결과를 df4에 저장합니다.

In [ ]:
df4= pd.get_dummies(df3, columns = ['상품명', '카테고리', '효과1', '효과2', '피부타입', '피부고민1', '피부고민2']).astype(int)
df4.head()

# 랜덤포레스트

#### 피처값과 타겟값을 지정해줍니다. (target : 별점, 피처는 별점 제외 모든 컬럼)

In [ ]:
features= df4.drop(columns = ['별점'])
target= df4['별점']

#### 저장한 피처와 타겟을 가지고 train : test 데이터를 8:2로 나누어 줍니다. random_state = 50

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#### df4의 별점의 분포를 막대그래프로 시각화하여 불균형한지 확인합니다.

In [ ]:
#한글 깨짐 방지
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

#막대그래프 코드 작성
count_star = df4['별점'].value_counts().sort_index()
plt.bar(count_star.index, count_star.values)
plt.xlabel("별점")
plt.ylabel("갯수")
plt.show()


### 불균형한 데이터 샘플링 SMOTE 활용

In [ ]:
#불균형 데이터 오버샘플링 SMOTE 
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

print("SMOTE 적용 전 train 데이터셋 shape : ", X_train.shape, y_train.shape)
print('SMOTE 적용 후 train 데이터셋 shape :', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 train 타겟 별점 분포',y_train_over.value_counts() )

### 데이터 스케일링
피처의 단위를 동일하게 만들기 위해 StandardScaler로 스케일링을 진행합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
                      
X_train_scaled = ss.fit_transform(X_train_over) 
X_test_scaled = ss.fit_transform(X_test)

### 그리드서치로 최적의 파라미터 찾기 
1. 지정할 파라미터와 값을 params에 저장합니다.  
2. 랜덤포레스트 모델을 생성하고 rfc로 할당합니다. (random_state=100, n_jobs=-1)  
3. 그리드서치를 실행하여 grid_cv로 할당합니다. (cv=3, n_jobs=-1)
4. 스케일링 완료한 훈련 데이터를 grid_cv에 fit해줍니다.
5. 최적의 파라미터를 print 합니다.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

#각 파라미터 값 넣기
params = { 'n_estimators' : [50, 100, 150],
           'max_depth' : [None, 10, 20],
           'max_features': ['auto', 'sqrt', 'log2'],
           'min_samples_split' : [2, 10, 20]
            }


rfc = RandomForestClassifier(random_state = 100, n_jobs = -1)
grid_cv = GridSearchCV(rfc, param_grid = params, cv = 3, n_jobs = -1)     # cv는 교차 검증을 위해 분할되는 학습 데이터의 세트 수
grid_cv.fit(X_train_scaled, y_train_over)

print(f'최적 파라미터: {grid_cv.best_params_}')

### 랜덤포레스트 모델 훈련
위에서 구한 최적 파라미터를 활용하여 랜덤포레스트 모델을 만들고 해당 모델로 다시 훈련시킵니다. random_state=100, n_jobs=-1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_param = {
    'max_depth' : 20,
    'max_features' : 'log2',
    'min_samples_split' : 2,
    'n_estimators' : 150
}
rfc= RandomForestClassifier(random_state=100, n_jobs=-1, **best_param)
rfc.fit(X_train_scaled, y_train_over)

 ### 모델 평가지표 결과값이 한번에 나오도록 함수 만들기
 함수에 y_test와 pred를 입력하면 정확도, 정밀도, 재현율, f1스코어가 데이터 프레임으로 나오도록 함수를 만듭니다.  
 단, 정확도, 정밀도, 재현율, f1스코어의 average는 모두 'micro'로 설정  
 딕셔너리를 활용하여 데이터 프레임을 만들어주세요.  

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score

def _metrics(y_test, pred):
    acc = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
    
    dict_metrics = {
        'Accuracy': [acc],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    }
    df_metrics = pd.DataFrame(dict_metrics)

    return df_metrics



#### 스케일링한 test데이터를 가지고 모델에 적용하여 나온 예측값을 pred에 할당하고 평가지표함수로 성능을 확인합니다.

In [ ]:
pred = rfc.predict(X_test_scaled)

_metrics(y_test,pred)

### 피처 중요도 확인하기

피처 중요도 값을 저장한 feat_importance 값을 내림차순하여 20개만 top_20에 할당합니다.  
그 값을 가지고 barplot을 만들어 줍니다. 
* 제목 : 피처 중요도 Top2 20 
* figsize = (8,6)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#제공 코드
feat_importance = pd.Series(rfc.feature_importances_, index=features.columns)

top20 = feat_importance.sort_values(ascending = False)[:20]

#한글 깨짐 방지
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False


plt.figure(figsize=(8,6))
plt.title('피처 중요도 Top2 20')
sns.barplot(x=top20.values, y=top20.index)
plt.show()

### 혼동행렬
y_test와 예측값 pred를 이용하여 혼동행렬 생성하여 cf에 할당합니다.

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(y_test, pred)
cf

### 혼동행렬 시각화
cf를 데이터 프레임(cf_matrix)으로 만든 후 컬럼명과, 인덱스를 1~5점으로 저장합니다.   
데이터 프레임을 heatmap으로 만들어 줍니다. ( anno=True  -> 각 셀에 숫자 입력 , fmt='d'  -> 정수형으로 나오도록함)   
x라벨 : 예측값, y라벨 : 실제값

In [ ]:
cf_matrix= pd.DataFrame(cf, index=range(1, 6), columns = range(1, 6))

#히트맵 코드 작성
plt.title('혼동행렬')
sns.heatmap(cf_matrix, annot = True, fmt = 'd', cmap = 'Blues', cbar = False)
plt.xlabel("예측값")
plt.ylabel("실제값")
plt.show()

# KNN

### 랜덤포레스트와 데이터 스케일링까지 과정이 반복됨

In [ ]:
#데이터 다시 확인해보기
df4.head()

#### 피처값과 타겟값  (target : 별점, 피처는 별점 제외 모든 컬럼)

In [ ]:
features=df4.drop('별점', axis = 1)
target= df4["별점"]

#### 저장한 피처와 타겟을 가지고 train : test 데이터를 8:2로 나누어 줍니다. random_state = 50

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

#train/test 데이터 분리 8:2로
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=50)

#### 불균형 데이터 smote로 오버샘플링

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

#### 피처의 단위를 동일하게 만들기 위해 StandardScaler로 스케일링을 진행합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler
      
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train_over)
X_test_scaled = ss.fit_transform(X_test)

### 그리드서치로 최적의 파라미터 찾기 
1. 지정할 파라미터와 값을 knn_params에 저장합니다.  
2. KNN 모델을 생성하고 knn으로 할당합니다. (n_jobs=-1)  
3. 그리드서치를 실행하여 grid_cv로 할당합니다. (cv=5, n_jobs=-1)
4. 스케일링 완료한 훈련 데이터를 grid_cv에 fit해줍니다.
5. 최적의 파라미터를 print 합니다.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

#각 파라미터 값 넣기
knn_params = {
    'n_neighbors' : [3, 5, 7],
    'weights' : ['uniform', 'distance']
}


knn= KNeighborsClassifier(n_jobs = -1)

grid_cv=GridSearchCV(knn, knn_params, cv = 5 , n_jobs = -1)       
grid_cv.fit(X_train_scaled,y_train_over)


print(f'최적 하이퍼 파라미터: {grid_cv.best_params_}')

### KNN 모델 훈련
위에서 구한 최적 파라미터를 활용하여 knn 모델을 만들고 해당 모델로 다시 훈련시킵니다. n_jobs=-1

In [ ]:
knn=KNeighborsClassifier(n_jobs=-1, n_neighbors=3, weights='uniform')
knn.fit(X_train_scaled,y_train_over)

#### 스케일링한 test데이터를 가지고 모델에 적용하여 나온 예측값을 knn_pred에 할당하고 평가지표함수로 성능을 확인합니다.

In [ ]:
knn_pred= knn.predict(X_test_scaled)

_metrics(y_test,knn_pred)

### 혼동행렬
y_test와 예측값 knn_pred를 이용하여 혼동행렬 생성하여 knn_cf에 할당합니다.

In [ ]:
from sklearn.metrics import confusion_matrix

knn_cf = confusion_matrix(y_test, knn_pred)
knn_cf

### 혼동행렬 시각화
knn_cf를 데이터 프레임(knn_cf_matrix)으로 만든 후 컬럼명과, 인덱스를 1~5점으로 저장합니다.   
데이터 프레임을 heatmap으로 만들어 줍니다. ( anno=True  -> 각 셀에 숫자 입력 , fmt='d'  -> 정수형으로 나오도록함)   
x라벨 : 예측값, y라벨 : 실제값

In [ ]:
knn_cf_matrix= pd.DataFrame(knn_cf, index = range(1, 6), columns = range(1, 6))

#히트맵 코드 작성
sns.heatmap(knn_cf_matrix, annot = True, fmt = 'd', cmap = 'reds', cbar = False)
plt.title("KNN 혼동행렬")
plt.xlabel("예측값")
plt.ylabel("실제값")
plt.show()